# Import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from collections import defaultdict

# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')

# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt

# We will use the Seaborn library
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'

## read data 


In [2]:
df = pd.read_csv("mHealth_subject3.log", header=None, delim_whitespace=True)
print(df)

             0         1         2         3         4          5        6   \
0       -9.4674  0.137210  1.054500  0.238620 -0.033490  -0.188480  -9.8425   
1       -9.5991  0.154810  1.182400  0.226060 -0.079540  -0.362630  -9.7832   
2       -9.8612  0.133170  1.064600  0.205130 -0.096285  -0.248230  -9.8010   
3       -9.6398  0.174060  1.236700  0.184200 -0.062794  -0.250270  -9.6730   
4      -10.0360  0.260620  1.276100  0.125590 -0.025118  -0.206680  -9.9102   
5       -9.6700  0.167460  0.857180  0.062794 -0.033490  -0.390890  -9.9108   
6       -9.2655  0.650970  1.395400  0.025118 -0.041863  -0.105080  -9.8407   
7       -9.6621  0.277490  0.972340  0.012559 -0.037677  -0.495030  -9.8329   
8       -9.4799  0.257660  1.117900  0.020931 -0.025118  -0.106760  -9.7624   
9       -9.6593  0.134870  1.096300  0.025118 -0.020931  -0.065413  -9.8618   
10      -9.6210  0.243950  1.337700  0.025118 -0.025118  -0.126850  -9.8621   
11      -9.6293  0.154090  1.224400  0.012559 -0.041

## check the samples size

In [3]:
for i in range(13): 
    df1=df[df[23] == i]
    print('number of samples of', i,"activity is", df1.shape )


number of samples of 0 activity is (86732, 24)
number of samples of 1 activity is (3072, 24)
number of samples of 2 activity is (3072, 24)
number of samples of 3 activity is (3072, 24)
number of samples of 4 activity is (3072, 24)
number of samples of 5 activity is (3072, 24)
number of samples of 6 activity is (3226, 24)
number of samples of 7 activity is (3379, 24)
number of samples of 8 activity is (3175, 24)
number of samples of 9 activity is (3072, 24)
number of samples of 10 activity is (3072, 24)
number of samples of 11 activity is (3072, 24)
number of samples of 12 activity is (1024, 24)


we found it better to exclude the labels with different size to end up with 8 classes rether than 12   

In [4]:
df=df[df[23] != 6]
df=df[df[23] != 7]
df=df[df[23] != 8]
df=df[df[23] != 12]
df=df[df[23] != 0]

## extract ECG column

In [5]:
def extractECG(dataFrame,lead=1):
    if lead == 1:
        lead = 4
    else :
        lead = 5
        
    data = dataFrame[lead]
#     data = data.reset_index()
#     data.columns=["index","readings"]
    return data
data=extractECG(df)

## combine labels based on frequency
for example for one seconds classification every 50 readings belongs to one label

In [6]:



readings=25
def fmax(label_count):
    M=0
    for i in range (int (len (label_count)) ):
        if label_count[i]>label_count[M]:
            M=i
    return M
labels=[]
label_index=0
label = df[23].values
label_count=[0,0,0,0,0,0,0,0]
for i in range(int(len(label))):
    if label[i]==1:
        label_count[0]=label_count[0]+1
    elif label[i]==2:
        label_count[1]=label_count[1]+1
    elif label[i]==3:
        label_count[2]=label_count[2]+1
    elif label[i]==4:
        label_count[3]=label_count[3]+1
    elif label[i]==5:
        label_count[4]=label_count[4]+1
    elif label[i]==9:
        label_count[5]=label_count[5]+1
    elif label[i]==10:
        label_count[6]=label_count[6]+1
    elif label[i]==11:
        label_count[7]=label_count[7]+1
    
    if(i%readings==readings-1):
        M=fmax(label_count)
        labels.append(M)
        label_count=[0,0,0,0,0,0,0,0,0,0,0,0,0]
        label_index=label_index+1
labels=np.array(labels)



## Extract features
based on some research we found that there are many features that can be measured from the ecg signal. we choose 2 features that are possible to be implemented.
* number of peaks in the signal
* heartrate[1] 

We will see if these 2 features are useful for prediction or not.

The following 2 blocks contain functions for features extraction

The heartrate is not measured to be beat/min. Instead it is dependent on number of readings we specify. As we know the data has a frequency of 50 readings/sec so when we use 25 readings the heartrate unit will be beat/halfsecond and so on    

In [7]:
def findmax(ecg):
    
    M=0
    
    index=0 
    for i in range(int(len(ecg))):
        if M < ecg[i]:
            M=ecg[i]
            index=i
        
    
    return index



def findmaxindex(ecg,peaks):
    
    M=0
    
    index=0
    for i in range(int(len(peaks))):
        if M < ecg[peaks[i]]:
            M=ecg[peaks[i]]
            index=i
        
    
    return index

def findpeak(ecg):
    
    start=0
    end=10
    
    peaks=[]
  
    for i in range(int(len(ecg)/10)):
        ecg1=ecg[start:end]
        ecg1=ecg1.reset_index(drop=True)
        peaks.append(findmax(ecg1))
        start=start+10
        end=end+10
        
    return peaks


In [8]:
# peaks1=findpeaksper2second(data[100:200])
# print(peaks1)



heartrate=[]
numbofpeaks=[]

def arrange(peaks):
    for i in range(int(len(peaks))):
        if peaks[i]!=-1 and peaks[i]!=0:
            peaks[i]=i*10+peaks[i]
            
    return peaks
    
for i in range (int(len(data)/readings)):
    data1=data[i*readings:(i+1)*readings]
    data1=data1.reset_index(drop=True)
    
    peaks1=findpeak(data1)
#     print (peaks1)
    maxindex=findmaxindex(data1,peaks1)
        
#     print(peaks1)    
    for j in range (int(len(peaks1))):
            
        if data1[peaks1[maxindex]]-data1[peaks1[j]]>0.7 :
            peaks1[j]=-1
#     print(peaks1)
    
    n=0
    for j in range (int(len(peaks1))):
        if peaks1[j]!=-1 or peaks1[j]!=0:
            n=n+1
    numbofpeaks.append(n)
    peaks1=arrange(peaks1)
#     print(peaks1)
#     break
    firstpeak=0
    secondpeak=0
    for j in range (int(len(peaks1))):
        if peaks1[j]!=-1 or peaks1[j]!=0:
            if firstpeak==0:
                firstpeak=peaks1[j]
            elif secondpeak==0:
                secondpeak=peaks1[j]
            else :
                break
    hr=abs(((firstpeak-secondpeak))/readings)
    heartrate.append(hr)

            
                

## prepare the ECG data

In [9]:
data=data.values
labels1=df[23].values
for j in range(data.size%readings):
    data = np.delete(data, (j), axis=0)
data= data.reshape(int(data.size/readings),1,readings)
numbofpeaks=np.array(numbofpeaks)
numbofpeaks= numbofpeaks.reshape(int(numbofpeaks.size),1,1)
heartrate=np.array(heartrate)
heartrate= heartrate.reshape(int(heartrate.size),1,1)

Now we have 3 types of data that can be used for models   


In [10]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

Using TensorFlow backend.


## spliting the data 

since it is a time series data it would be better to split the data by taking the first big part for training and the last small part for validation and testing. However, this can't be done because the last part of the data doesn't have the labels that the first part has. so different classes might appear in the testing set. Therefore, we used split function. 

In [11]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(heartrate, to_categorical(labels), test_size = 0.20, random_state = 0)



## heartrate 1D CNN model

In [12]:
verbose, epochs, batch_size = 0, 10, 32
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], 8
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', 
                   input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics=['accuracy'])

 


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
X_train.shape
model.summary()
y_train.shape

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1, 64)             128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 64)             4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               33280     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 4104      
Total para

(786, 8)

In [14]:
# X_train = X_train.reshape((392, 1, 50))

# fit network
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
# evaluate model


Instructions for updating:
Use tf.cast instead.
Epoch 1/100
786/786 [==============================] - 2s 2ms/step - loss: 2.0813 - acc: 0.1310
Epoch 2/100
786/786 [==============================] - 0s 53us/step - loss: 2.0758 - acc: 0.1336
Epoch 3/100
786/786 [==============================] - 0s 49us/step - loss: 2.0724 - acc: 0.1641
Epoch 4/100
786/786 [==============================] - 0s 55us/step - loss: 2.0685 - acc: 0.1578
Epoch 5/100
786/786 [==============================] - 0s 58us/step - loss: 2.0631 - acc: 0.1489
Epoch 6/100
786/786 [==============================] - 0s 52us/step - loss: 2.0581 - acc: 0.1438
Epoch 7/100
786/786 [==============================] - 0s 55us/step - loss: 2.0516 - acc: 0.1298
Epoch 8/100
786/786 [==============================] - 0s 57us/step - loss: 2.0424 - acc: 0.1654
Epoch 9/100
786/786 [==============================] - 0s 54us/step - loss: 2.0443 - acc: 0.1616
Epoch 10/100
786/786 [==============================] - 0s 61us/step - loss: 2.0

Epoch 82/100
786/786 [==============================] - 0s 53us/step - loss: 1.9992 - acc: 0.1972
Epoch 83/100
786/786 [==============================] - 0s 61us/step - loss: 2.0035 - acc: 0.1819
Epoch 84/100
786/786 [==============================] - 0s 60us/step - loss: 1.9976 - acc: 0.1819
Epoch 85/100
786/786 [==============================] - 0s 72us/step - loss: 1.9961 - acc: 0.1883
Epoch 86/100
786/786 [==============================] - 0s 60us/step - loss: 1.9865 - acc: 0.1959
Epoch 87/100
786/786 [==============================] - 0s 63us/step - loss: 1.9984 - acc: 0.2010
Epoch 88/100
786/786 [==============================] - 0s 53us/step - loss: 1.9977 - acc: 0.1870
Epoch 89/100
786/786 [==============================] - 0s 55us/step - loss: 1.9971 - acc: 0.1870
Epoch 90/100
786/786 [==============================] - 0s 49us/step - loss: 2.0026 - acc: 0.1858
Epoch 91/100
786/786 [==============================] - 0s 51us/step - loss: 1.9993 - acc: 0.1934
Epoch 92/100
786/786

In [15]:
accuracy = model.evaluate(X_test, y_test, batch_size=32, verbose=1)
print (accuracy)

197/197 [==============================] - 0s 264us/step
[2.009075570227531, 0.1319796954314721]


## Number of peaks model

In [16]:
X_train, X_test, y_train, y_test = train_test_split(numbofpeaks, to_categorical(labels), test_size = 0.20, random_state = 0)

print(X_train.shape)
print(y_train.shape)

verbose, epochs, batch_size = 0, 10, 32
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], 8
model_numofpeaks = Sequential()
model_numofpeaks.add(Conv1D(filters=64, kernel_size=1, activation='relu', 
                   input_shape=(n_timesteps,n_features)))
model_numofpeaks.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model_numofpeaks.add(Dropout(0.5))
model_numofpeaks.add(MaxPooling1D(pool_size=1))
model_numofpeaks.add(Flatten())
model_numofpeaks.add(Dense(512, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
model_numofpeaks.add(Dense(n_outputs, activation='softmax'))
model_numofpeaks.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics=['accuracy'])

model_numofpeaks.summary()
model_numofpeaks.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
accuracy = model_numofpeaks.evaluate(X_test, y_test, batch_size=32, verbose=1)
print (accuracy)



(786, 1, 1)
(786, 8)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 1, 64)             128       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1, 64)             4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               33280     
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 

786/786 [==============================] - 0s 295us/step - loss: 2.0779 - acc: 0.1374
Epoch 71/100
786/786 [==============================] - 0s 397us/step - loss: 2.0782 - acc: 0.1374
Epoch 72/100
786/786 [==============================] - 0s 413us/step - loss: 2.0781 - acc: 0.1374
Epoch 73/100
786/786 [==============================] - 0s 329us/step - loss: 2.0779 - acc: 0.1374
Epoch 74/100
786/786 [==============================] - 0s 277us/step - loss: 2.0782 - acc: 0.1374
Epoch 75/100
786/786 [==============================] - 0s 488us/step - loss: 2.0781 - acc: 0.1374
Epoch 76/100
786/786 [==============================] - 0s 417us/step - loss: 2.0778 - acc: 0.1374
Epoch 77/100
786/786 [==============================] - 0s 284us/step - loss: 2.0781 - acc: 0.1374
Epoch 78/100
786/786 [==============================] - 0s 200us/step - loss: 2.0779 - acc: 0.1374
Epoch 79/100
786/786 [==============================] - 0s 576us/step - loss: 2.0782 - acc: 0.1374
Epoch 80/100
786/786 [=

## ECG data model

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data, to_categorical(labels), test_size = 0.20, random_state = 0)

print(X_train.shape)
print(y_train.shape)

verbose, epochs, batch_size = 0, 10, 32
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], 8
model_data = Sequential()
model_data.add(Conv1D(filters=64, kernel_size=1, activation='relu', 
                   input_shape=(n_timesteps,n_features)))
model_data.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model_data.add(Dropout(0.5))
model_data.add(MaxPooling1D(pool_size=1))
model_data.add(Flatten())
model_data.add(Dense(512, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
model_data.add(Dense(n_outputs, activation='softmax'))
model_data.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics=['accuracy'])

model_data.summary()
model_data.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
accuracy = model_data.evaluate(X_test, y_test, batch_size=32, verbose=1)
print (accuracy)


(786, 1, 25)
(786, 8)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 1, 64)             1664      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1, 64)             4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               33280     
_________________________________________________________________
dense_4 (Dense)              (None, 8)                

786/786 [==============================] - 0s 59us/step - loss: 0.5347 - acc: 0.8079
Epoch 72/100
786/786 [==============================] - 0s 56us/step - loss: 0.5110 - acc: 0.8104
Epoch 73/100
786/786 [==============================] - 0s 57us/step - loss: 0.5636 - acc: 0.8003
Epoch 74/100
786/786 [==============================] - 0s 58us/step - loss: 0.5306 - acc: 0.7952
Epoch 75/100
786/786 [==============================] - 0s 56us/step - loss: 0.5370 - acc: 0.8028
Epoch 76/100
786/786 [==============================] - 0s 53us/step - loss: 0.5287 - acc: 0.7952
Epoch 77/100
786/786 [==============================] - 0s 55us/step - loss: 0.5165 - acc: 0.7964
Epoch 78/100
786/786 [==============================] - 0s 58us/step - loss: 0.5363 - acc: 0.7990
Epoch 79/100
786/786 [==============================] - 0s 55us/step - loss: 0.5021 - acc: 0.8193
Epoch 80/100
786/786 [==============================] - 0s 53us/step - loss: 0.4922 - acc: 0.8155
Epoch 81/100
786/786 [===========

## Another try with scaling 

In [17]:
from sklearn.preprocessing import StandardScaler
def scale_data(trainX, testX, standardize):
	# remove overlap
	cut = int(trainX.shape[1] / 2)
	longX = trainX[:, -cut:, :]
	# flatten windows
	longX = longX.reshape((longX.shape[0] * longX.shape[1], longX.shape[2]))
	# flatten train and test
	flatTrainX = trainX.reshape((trainX.shape[0] * trainX.shape[1], trainX.shape[2]))
	flatTestX = testX.reshape((testX.shape[0] * testX.shape[1], testX.shape[2]))
	# standardize
	if standardize:
		s = StandardScaler()
		# fit on training data
		s.fit(longX)
		# apply to training and test data
		longX = s.transform(longX)
		flatTrainX = s.transform(flatTrainX)
		flatTestX = s.transform(flatTestX)
	# reshape
	flatTrainX = flatTrainX.reshape((trainX.shape))
	flatTestX = flatTestX.reshape((testX.shape))
	return flatTrainX, flatTestX

In [18]:
trainX, testX = scale_data(X_train, X_test, True)

In [19]:
verbose, epochs, batch_size = 0, 10, 32
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], 8
model1 = Sequential()
model1.add(Conv1D(filters=128, kernel_size=1, activation='relu', 
                   input_shape=(n_timesteps,n_features)))
model1.add(BatchNormalization())
model1.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(MaxPooling1D(pool_size=1))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(BatchNormalization())
# model1.add(Dense(64, activation='relu'))
# model1.add(BatchNormalization())
model1.add(Dense(n_outputs, activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics=['accuracy'])

In [20]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 1, 128)            3328      
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 128)            512       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1, 64)             8256      
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 64)             256       
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 64)                0         
__________

In [21]:
model1.fit(trainX, y_train, epochs=150, batch_size=32, verbose=1)

Epoch 1/150
786/786 [==============================] - 1s 1ms/step - loss: 2.5101 - acc: 0.1578
Epoch 2/150
786/786 [==============================] - 0s 69us/step - loss: 2.0295 - acc: 0.2341
Epoch 3/150
786/786 [==============================] - 0s 71us/step - loss: 1.9192 - acc: 0.2748
Epoch 4/150
786/786 [==============================] - 0s 65us/step - loss: 1.7491 - acc: 0.3168
Epoch 5/150
786/786 [==============================] - 0s 65us/step - loss: 1.6972 - acc: 0.3333
Epoch 6/150
786/786 [==============================] - 0s 68us/step - loss: 1.6568 - acc: 0.3511
Epoch 7/150
786/786 [==============================] - 0s 65us/step - loss: 1.6146 - acc: 0.3842
Epoch 8/150
786/786 [==============================] - 0s 66us/step - loss: 1.5663 - acc: 0.3919
Epoch 9/150
786/786 [==============================] - 0s 62us/step - loss: 1.5200 - acc: 0.3982
Epoch 10/150
786/786 [==============================] - 0s 64us/step - loss: 1.5300 - acc: 0.3957
Epoch 11/150
786/786 [========

786/786 [==============================] - 0s 85us/step - loss: 0.7769 - acc: 0.7201
Epoch 85/150
786/786 [==============================] - 0s 65us/step - loss: 0.7863 - acc: 0.7099
Epoch 86/150
786/786 [==============================] - 0s 62us/step - loss: 0.8306 - acc: 0.6858
Epoch 87/150
786/786 [==============================] - 0s 63us/step - loss: 0.7901 - acc: 0.6781
Epoch 88/150
786/786 [==============================] - 0s 92us/step - loss: 0.7622 - acc: 0.6959
Epoch 89/150
786/786 [==============================] - 0s 69us/step - loss: 0.7655 - acc: 0.7137
Epoch 90/150
786/786 [==============================] - 0s 64us/step - loss: 0.7846 - acc: 0.6743
Epoch 91/150
786/786 [==============================] - 0s 66us/step - loss: 0.7361 - acc: 0.7163
Epoch 92/150
786/786 [==============================] - 0s 81us/step - loss: 0.7589 - acc: 0.6921
Epoch 93/150
786/786 [==============================] - 0s 88us/step - loss: 0.7431 - acc: 0.7061
Epoch 94/150
786/786 [===========

In [22]:
accuracy = model1.evaluate(testX, y_test, batch_size=32, verbose=1)
print (accuracy)

197/197 [==============================] - 0s 809us/step
[1.491453538691332, 0.5329949240091488]


## Conclusion
* features extraction was not usefull as we have seen. we noticed from the data behaviour that the value of the peaks matters more than the number of peaks.(to be considered in further work)
* ECG data was giving acuraccy of 58% at some moments which is better than the other trials. but still we will try to increase the accuracy in 2D CNN   

# References 
* TuanNguyen GiaaImed Ben Dhaoub Mai Alic Amir M.Rahmanide Tomi Wester lunda Pasi Liljeberga Hannu Tenhunena. “Energy efficient fog-assisted IoT system for monitoring diabetic patients with cardiovascular disease”